In [ ]:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!pip install wandb -qU

# Log in to your W&B account
import wandb
import random
import math

wandb.login()


In [ ]:
# List contents of your Drive
#import os
#os.listdir('/content/drive/My Drive/')

# Load a file from your Drive
#with open('/content/drive/My Drive/FMRIR/fm_utils.py', 'r') as f:
    #content = f.read()

# Save a file to your Drive
#with open('/content/drive/My Drive/new_file.txt', 'w') as f:
    #f.write('This is a new file saved from Colab.')

import sys
sys.path.append('/content/drive/My Drive/FMRIR')
sys.path.append('/content/drive/My Drive')
import torch
from torchvision import transforms
import os
import json
import time
import argparse

from fm_utils import (
    SpectrogramSampler, GaussianConditionalProbabilityPath, LinearAlpha,
    LinearBeta, CFGTrainer, SpecUNet
)


['Başlıksız form.gform',
 'IS100-ProfExam-Monday.xlsx',
 'IS100-ProfExam-Monday.gsheet',
 '369bce03.png',
 'Yurt İçi ve Yurt Dışında Öğretim Elemanı ve Araştırmacı Destekleme Programlarına İlişkin Usul ve Esaslar.pdf',
 '2202304-3329.pdf',
 'M2192- Use Cases and Functional Requirements - Six DoF Audio (CAE-6DA) WD0.1.1.docx',
 'cool, what changes do we need for u-net? preservi...',
 'ir_fs2000_s1024_m1331_room4.0x6.0x3.0_rt200',
 'Colab Notebooks',
 'FMRIR']

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Configuration ---
config = {
    "data": {
        "data_dir": "/content/drive/My Drive/ir_fs2000_s1024_m1331_room4.0x6.0x3.0_rt200/",
        "src_splits": {
            "train": [0, 820],
            "valid": [820, 922],
            "test": [922, 1024],
            "all": [0, 1024]}
    },
    "model": {
        "name": "SpecUNet",
        "channels": [32, 64, 128],
        "num_residual_layers": 2,
        "t_embed_dim": 40,
        "y_dim": 6,
        "y_embed_dim": 40
    },
    "training": {
        "num_epochs": 10,
        "batch_size": 250,
        "lr": 1e-3,
        "eta": 0.1
    },
    "experiments_dir": "/content/drive/MyDrive/FMRIR"
}

# --- Experiment Setup ---
timestamp = time.strftime("%Y%m%d-%H%M%S")
experiment_name = f"{config['model']['name']}_{timestamp}"
experiment_dir = os.path.join(config['experiments_dir'], experiment_name)
os.makedirs(experiment_dir, exist_ok=True)

MODEL_SAVE_PATH = os.path.join(experiment_dir, "model.pt")
#CONFIG_SAVE_PATH = os.path.join(experiment_dir, "config.json")



In [33]:
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="ege-erdem-king-s-college-london",
    # Set the wandb project where this run will be logged.
    project="FM-RIR",
    # Track hyperparameters and run metadata.
    config=config
    )

In [34]:
# --- Data Loading ---
data_cfg = config['data']
temp_sampler = SpectrogramSampler(data_path=data_cfg['data_dir'], mode="all", src_splits=data_cfg['src_splits'])

spec_mean = temp_sampler.spectrograms.mean()
spec_std = temp_sampler.spectrograms.std()

spec_train_sampler = SpectrogramSampler(
    data_path=data_cfg['data_dir'], mode='train', src_splits=data_cfg['src_splits'],
    transform=transforms.Compose([transforms.Normalize((spec_mean,), (spec_std,))])
).to(device)

sample_spec, _ = spec_train_sampler.sample(1)
spec_shape = list(sample_spec.shape[1:])

path = GaussianConditionalProbabilityPath(
    p_data=spec_train_sampler, #was [1, 32, 32], for mnist
    p_simple_shape=spec_shape,
    alpha=LinearAlpha(),
    beta=LinearBeta()
).to(device)


# --- Model and Trainer Initialization ---
model_cfg = config['model']
training_cfg = config['training']

spec_unet = SpecUNet(
    channels=model_cfg['channels'], # Same as MNIST version
    num_residual_layers=model_cfg['num_residual_layers'], # Same as MNIST version
    t_embed_dim=model_cfg['t_embed_dim'], # Same as MNIST version
    y_dim=model_cfg['y_dim'], # new: 6D coordinates for source and microphone positions
    y_embed_dim=model_cfg['y_embed_dim'], # Same as MNIST version
).to(device)

trainer = CFGTrainer(
    path=path,
    model=spec_unet,
    eta=training_cfg['eta'],
    y_dim=model_cfg['y_dim'],
)

# --- Training ---
print(f"--- Starting Training for experiment: {experiment_name} ---")
trainer.train(
    num_epochs=training_cfg['num_epochs'],
    device=device,
    lr=training_cfg['lr'],
    batch_size=training_cfg['batch_size']
)

# --- Save the Model ---
print(f"Saving model to {MODEL_SAVE_PATH}...")
torch.save({
    'model_state_dict': spec_unet.state_dict(),
    'y_null': trainer.y_null,
    'config': config # Save config with model for easy reference
}, MODEL_SAVE_PATH)
print("Model saved. You can now run inference using the model and config from the experiment directory.")
print(f"Experiment directory: {experiment_dir}")

Loading pre-processed all data from /content/drive/My Drive/ir_fs2000_s1024_m1331_room4.0x6.0x3.0_rt200/processed_all.pt
Loaded 21296 spectrograms for all set.
Spectrogram tensor shape: torch.Size([21296, 16, 16])
Coordinate tensor shape: torch.Size([21296, 6])
Loading pre-processed train data from /content/drive/My Drive/ir_fs2000_s1024_m1331_room4.0x6.0x3.0_rt200/processed_train.pt
Loaded 1091420 spectrograms for train set.
Spectrogram tensor shape: torch.Size([1091420, 16, 16])
Coordinate tensor shape: torch.Size([1091420, 6])
--- Starting Training for experiment: SpecUNet_20250730-094954 ---
Training model with size: 4.721 MiB


Epoch 1, loss: 787.228: : 2it [00:02,  1.06it/s]

Epoch 0, loss: 639.167
Epoch 1, loss: 787.228


Epoch 3, loss: 614.428: : 4it [00:02,  2.58it/s]

Epoch 2, loss: 1124.957
Epoch 3, loss: 614.428


Epoch 6, loss: 402.356: : 6it [00:02,  4.28it/s]

Epoch 4, loss: 539.184
Epoch 5, loss: 412.349
Epoch 6, loss: 402.356


Epoch 9, loss: 375.009: : 10it [00:03,  3.29it/s]


Epoch 7, loss: 391.082
Epoch 8, loss: 372.250
Epoch 9, loss: 375.009
Saving model to /content/drive/MyDrive/FMRIR/SpecUNet_20250730-094954/model.pt...
Model saved. You can now run inference using the model and config from the experiment directory.
Experiment directory: /content/drive/MyDrive/FMRIR/SpecUNet_20250730-094954
